In [1]:
import FinanceDataReader as fdr
import requests
import pandas as pd
import time
import ta
import openpyxl
import warnings
import cloudscraper
import tensorflow as tf
tf.config.experimental.set_visible_devices([], 'GPU')

from datetime import datetime, timedelta
from pykrx import stock
from pykrx import bond
from PublicDataReader import Fred
from marcap import marcap_data
from functools import reduce
from collections import OrderedDict
from defs_pred import Marcap, filter_df
from io import BytesIO
from openpyxl.utils.dataframe import dataframe_to_rows
from ics import Calendar, Event

In [2]:
targer_day = datetime(year=2024,month=3,day=7)
end_info_day = (targer_day - timedelta(days=1))
delta = timedelta(days=375)
delta2 = timedelta(days=60)
day_21 = (end_info_day - delta2).strftime("%Y-%m-%d")
day_120 = (end_info_day - delta).strftime("%Y-%m-%d")


a = stock.get_market_ticker_list("20240306", market="KOSPI")
b = stock.get_market_ticker_list("20240306", market="KOSDAQ")
kospi = pd.Series(a)
kosdaq = pd.Series(b)
code = pd.concat([kospi,kosdaq],axis=0)

df = pd.DataFrame(code, columns=['Code'])
high_52_week_list = []
change_percentage_list = []
average_volume_5_days = []

for stock_code in code:
# 10일 전 날짜 계산
    high_52_date = (end_info_day - timedelta(days=375)).strftime('%Y-%m-%d')

    # FinanceDataReader를 사용하여 주식 데이터 가져오기
    stock_data = fdr.DataReader(stock_code, start=high_52_date, end=end_info_day).reset_index()
    stock_data['Volume'] = stock_data['Volume'] * ((stock_data['High'] + stock_data['Low']) / 2)

    # 52주 최고가 계산
    rolling_result = stock_data['High'].rolling(window=250).max()
    
    # 최근 종가 대비 52주 최고가 대비 변동율 계산
    change_percentage = round((stock_data['Close'].shift(1) - rolling_result) / rolling_result, 3)

    # 거래대금 평균 계산
    stock_data = stock_data.tail(5)
    average_volume = int(stock_data['Volume'].mean())
    
    high_52_week_list.append(rolling_result.tail(1))
    change_percentage_list.append(change_percentage.tail(1))
    average_volume_5_days.append(average_volume)
    
df['52주 최고가'] = high_52_week_list
df['52주 최고가 대비 변동율'] = change_percentage_list
df['5일 평균 거래대금'] = average_volume_5_days

In [4]:
result_df = df[df['5일 평균 거래대금'] >= 50000000000]
result_df['52주 최고가 대비 변동율'] = result_df['52주 최고가 대비 변동율'].apply(lambda x: float(x))

# 변동율을 기준으로 내림차순 정렬
sorted_df = result_df.sort_values(by='52주 최고가 대비 변동율', ascending=True)
 
# 상위 100개만 선택하여 새로운 데이터프레임 생성
top_100_df = sorted_df.head(200)
top_100_df = top_100_df.drop(['52주 최고가','52주 최고가 대비 변동율','5일 평균 거래대금'],axis=1)
top_100_list = top_100_df['Code'].tolist()

/var/folders/j9/2n544hl17x51jqvjb72jz9s40000gp/T/ipykernel_30853/139744197.py:2: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  result_df['52주 최고가 대비 변동율'] = result_df['52주 최고가 대비 변동율'].apply(lambda x: float(x))
/var/folders/j9/2n544hl17x51jqvjb72jz9s40000gp/T/ipykernel_30853/139744197.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['52주 최고가 대비 변동율'] = result_df['52주 최고가 대비 변동율'].apply(lambda x: float(x))
